# ERA-5 Quantile

In [ ]:
%%capture
!pip install netcdf4 colorcet boto3 urllib3==1.25.10
import os, fnmatch, gzip, shutil, datetime, numpy as np, pandas as pd, xarray as xr
import pylab as plt, plotly.express as px, seaborn as sns, colorcet as cc, altair as alt
from matplotlib.gridspec import GridSpec; from matplotlib.colors import from_levels_and_colors as flc
from metpy.units import units; import metpy.calc as mpcalc
import boto3, botocore

#!pip install --upgrade zarr gcsfs cftime nc-time-axis
#import fsspec, zarr, gcsfs; gcs = gcsfs.GCSFileSystem( token = 'anon' )

%load_ext google.colab.data_table 
from google.colab import drive; drive.mount( '/content/drive' )

## download ERA calc THI by two specifications, save arrays to drive

RH from bolton 1980 (http://www.eol.ucar.edu/projects/ceop/dm/documents/refdata_report/eqns.html)

In [ ]:
era5_bucket = 'era5-pds'
client = boto3.client( 's3', config = botocore.client.Config( signature_version=botocore.UNSIGNED ) )

In [ ]:
years = np.arange(1985,2015,1)
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

for year in years:
    for month in months:
        print(year, month)
        client.download_file( era5_bucket, f'{year}/{month}/data/air_temperature_at_2_metres.nc', 
                            f'/content/{year}_{month}_air_temperature_at_2_metres.nc' )

        client.download_file( era5_bucket, f'{year}/{month}/data/dew_point_temperature_at_2_metres.nc', 
                            f'/content/{year}_{month}_dew_point_temperature_at_2_metres.nc' )

        Ta = xr.open_dataarray(f'/content/{year}_{month}_air_temperature_at_2_metres.nc') - 273.15
        Tdp = xr.open_dataarray(f'/content/{year}_{month}_dew_point_temperature_at_2_metres.nc') - 273.15
        RH  = np.exp( ( 17.625 * Tdp ) / ( 243.04 + Tdp ) ) / np.exp( ( 17.625 * Ta ) / ( 243.04 + Ta ) )
        RH = xr.where(RH < 1, RH, 1)
        
        thi = (1.8 * Ta + 32) - ( (0.55 - 0.55 * RH) * (1.8 * Ta - 26) )
        thim = thi.resample( time0 = '1D' ).max()
        thim.to_dataset(name='THI').to_netcdf(f'/content/drive/My Drive/data/livestock/ERA5/max/{year}_{month}_daily_max_thi.nc')
        thim = thi.resample( time0 = '1D' ).mean()
        thim.to_dataset(name='THI').to_netcdf(f'/content/drive/My Drive/data/livestock/ERA5/mean/{year}_{month}_daily_mean_thi.nc')
        
        del thi 

        os.remove(f'/content/{year}_{month}_air_temperature_at_2_metres.nc')
        os.remove(f'/content/{year}_{month}_dew_point_temperature_at_2_metres.nc')




---

# OLD

---



In [ ]:
keys = []
date = datetime.date( 1985, 1, 1 ) # update to desired date
prefix = date.strftime( '%Y/%m/' )

response = client.list_objects_v2( Bucket=era5_bucket, Prefix=prefix )
response_meta = response.get( 'ResponseMetadata' )

if response_meta.get('HTTPStatusCode') == 200:
    contents = response.get('Contents')
    if contents == None: print("No objects are available for %s" % date.strftime('%B, %Y'))
    else:
        for obj in contents: keys.append(obj.get('Key'))
        print( "There are %s objects available for %s\n--" % (len(keys), date.strftime('%B, %Y')))
        for k in keys: print(k)
else: print( "There was an error with your request." )

In [ ]:
ds = xr.open_dataarray(f'/content/{year}_{month}_air_temperature_at_2_metres.nc')
ds.metpy.convert_units('degC')

dsd = xr.open_dataarray(f'/content/{year}_{month}_dew_point_temperature_at_2_metres.nc')
dsd.metpy.convert_units('degC')

rh = mpcalc.relative_humidity_from_dewpoint( ds , dsd ) 

In [ ]:
### Reference ###

In [ ]:
def THI(spec, Ta, Tdp = False, huss = False):
    # takes dry-bulb temperature in C and either dewpoint (C) or specific humidity (kg/kg)
    # returns THI based on whichever specificciation you choose
    Ta = np.nan_to_num( np.atleast_1d( Ta ) )
    temp  = Ta * units.degC
    press = np.array( 1e5 ) * units.hectopascal

    if huss == False: 
        Tdp = np.nan_to_num( np.atleast_1d( Tdp ) )
        dew = Tdp * units.degC 
        RH  = np.asarray( mpcalc.relative_humidity_from_dewpoint( temp , dew ) )
        ### Wet bulb from Stull, 2011 Journal of Applied Meterology and Climatology, Equation 1
        #Twb = Ta * np.arctan( 0.151977 * (RH + 8.313659)**(1/2) ) + np.arctan(Ta+ RH) - np.arctan(RH - 1.676331) + 0.00391838 * (RH)**(3/2) * np.arctan(0.023101 * RH) - 4.686035

    elif Tdp == False:
        huss = np.nan_to_num( np.atleast_1d( huss ) )
        SH  =  huss * units.dimensionless
        RH  = np.asarray( mpcalc.relative_humidity_from_specific_humidity(SH, temp, press ) )
        Tdp = np.asarray( mpcalc.dewpoint_from_specific_humidity( SH, temp, press ) )

    if   spec == 'thom':   thi = 0.80*Ta + Ta*RH - 1.4*RH + 46.4      ## adapted from Thom59, they do something goofy with the conversion
    elif spec == 'kibler': thi = 1.00*Ta + 0.36*Tdp + 41.2            ## Kibler 64, NRC71, then yousef85

    thi = np.ma.masked_outside( np.nan_to_num( thi ), 1, 200 ).compressed()

    return thi

In [ ]:
ds.temperature.metpy.convert_units('degC')
ds.

thiT = 0.80*Ta + Ta*RH - 1.4*RH + 46.4      
thiK = 1.00*Ta + 0.36*Tdp + 41.2            


In [ ]:
!pip install --upgrade botocore s3fs

In [ ]:
import datetime
from dask.distributed import LocalCluster, Client
import s3fs
import boto3

In [ ]:
import botocore

In [ ]:
era5_bucket = 'era5-pds'
client = boto3.client('s3', config=botocore.client.Config(signature_version=botocore.UNSIGNED))

In [ ]:
keys = []
date = datetime.date( 2019, 1, 1 ) # update to desired date
prefix = 'zarr/2008/01/data/'  

response = client.list_objects_v2( Bucket=era5_bucket, Prefix=prefix )
response_meta = response.get( 'ResponseMetadata' )

if response_meta.get('HTTPStatusCode') == 200:
    contents = response.get('Contents')
    if contents == None: print("No objects are available for %s" % date.strftime('%B, %Y'))
    else:
        for obj in contents: keys.append(obj.get('Key'))
        print( "There are %s objects available for %s\n--" % (len(keys), date.strftime('%B, %Y')))
        for k in keys: print(k)
else: print( "There was an error with your request." )

In [ ]:
fs = s3fs.S3FileSystem(anon=True)

In [ ]:
def inc_mon(indate):
    if indate.month < 12:
        return datetime.datetime(indate.year, indate.month+1, 1)
    else:
        return datetime.datetime(indate.year+1, 1, 1)

def gen_d_range(start, end):
    rr = []
    while start <= end:
        rr.append(start)
        start = inc_mon(start)
    return rr

def get_z(dtime,var):
    f_zarr = 'era5-pds/zarr/{year}/{month:02d}/data/{var}.zarr/'.format(year=dtime.year, month=dtime.month,var=var)
    return xr.open_zarr(s3fs.S3Map(f_zarr, s3=fs))

def gen_zarr_range(start, end,var):
    return [get_z(tt,var) for tt in gen_d_range(start, end)]

In [ ]:
tmp_a = gen_zarr_range(datetime.datetime(1980,1,1), datetime.datetime(1980,3,31),'air_temperature_at_2_metres')
tmp_all = xr.concat(tmp_a, dim='time0')
tmp_all = tmp_all.air_temperature_at_2_metres.chunk({'time0': -1}).resample(time0='1D').max()
tmp_all = tmp_all.quantile( [0.9,0.95], dim = 'time0' )
tmp_all.to_netcdf('/content/drive/My Drive/data/livestock/t2m_90_quantile.nc4')

In [ ]:
resample(time0='1D').quantile( [0.9,0.95], dim = 'time0' )

In [ ]:
tmp_all

# Climate data

In [ ]:
ds = xr.open_zarr( fsspec.get_mapper('gcs://pangeo-era5/reanalysis/temporal-analysis'), 
                  consolidated = True, chunks = 'auto')# {'time': -1, 'latitude':10, 'longitude':10}  )
ds

In [ ]:
path = '/content/drive/My Drive/data/livestock/t2m_90_quantile.nc4'
quants = [0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99]
ds.t2m.quantile( quants, dim = 'time' ).to_netcdf(path)

In [ ]:
path = '/content/drive/My Drive/data/livestock/1984_d2m_90up_quantile.nc4'
quants = [0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99]
with ProgressBar():
    ds.d2m.sel( time = slice('1984-01-01T00:00:00', '1984-01-01T23:00:00') ).to_netcdf(path)

In [ ]:
d = ds.t2m.sel( time = '1984-01-01T00:00:00' ).values

In [ ]:
ds.t2m.sel(time='2000-12-31T23:00:00').squeeze().plot()

In [ ]:
d = d.to_netcdf(path)

In [ ]:
path = '/content/drive/My Drive/data/livestock/1984_d2m_90up_quantile.nc4'
quants = [0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99]
d = ds.t2m.sel(time = slice('1984-01-01T00:00:00', '1984-01-31T23:00:00')).quantile( quants, dim = 'latitude' )

with ProgressBar():
    out = d.compute()

In [ ]:
path = '/content/drive/My Drive/data/livestock/t2m_90_quantile.nc4'
quants = [0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99]
ds.t2m.sel( latitude = slice(90,87.5) ).resample( time = '1D' ).max().quantile( quants, dim = 'time' ).to_netcdf(path)

### CMIP6

In [ ]:
df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv' )
df = df.query( "activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == 'ssp585' & source_id == 'IPSL-CM6A-LR' & table_id == 'day'" )
df

In [ ]:
ds = xr.open_zarr( gcs.get_mapper( df.zstore.values[0] ), consolidated = True )

In [ ]:
import time
import xarray as xr
import numpy as np

from tqdm import tqdm_notebook
from xarray.core.groupby import DataArrayGroupBy, DatasetGroupBy

def inner_generator(df_function='apply'):
    def inner(df, func, *args, **kwargs):
        
        t = tqdm_notebook(total = len(df))
        def wrapper(*args, **kwargs):
            t.update( n=1 if not t.total or t.n < t.total else 0 )
            return func( *args, **kwargs )

        result = getattr(df, df_function)(wrapper, **kwargs)

        t.close()
        return result
    return inner

DataArrayGroupBy.progress_apply = inner_generator()
DatasetGroupBy.progress_apply = inner_generator()

arr = xr.DataArray( np.arange( 10 ), dims = [ 'x' ] )

def foo( v ):
    time.sleep( 1 )
    return v + 15

print( arr.groupby( 'x' ).progress_apply( foo ) )